In [1]:
import time
import os
import numpy as np
from pynq import allocate
from pynq import MMIO
from pynq import Overlay

In [2]:
os.getcwd()
ol = Overlay("dma_loopback.bit")

In [3]:
# base addr
DMA_GRABBER = 0x40000000
GPIO_FIFO   = 0x41200000

# GPIO
remain_data_num   = 0x10
dma_garbber_en    = 0x10
dma_garbber_size  = 0x18

In [4]:
# 写寄存器
def write_reg(base_addr, offset, val):
    mmio = MMIO(base_addr, 64*1024)   # 创建一个名为mimo的MMIO对象,可用地址空间为64KB(与VIVADO设计中的地址分配相吻合)
    mmio.write(offset, val)           # 向地址为base_addr+offset的寄存器写入val


# 读寄存器
def read_reg(base_addr, offset):  
    mmio = MMIO(base_addr, 64*1024)
    read_val = mmio.read(offset)
    return read_val


# 写DMA
def dma_write(data):
    dma = ol.axi_dma_0
    data_in = allocate(shape=(len(data),), dtype=np.uint32)
    np.copyto(data_in, data)
    dma.sendchannel.transfer(data_in)
    dma.sendchannel.wait()
    data_in.close()
    tot = read_reg(GPIO_FIFO, remain_data_num) + 4


# 读DMA(read_size=0时,读出当前DMA中的全部数据)
def dma_read(read_size):
    dma = ol.axi_dma_0
    tot = read_reg(GPIO_FIFO, remain_data_num) + 2
    size = tot if read_size == 0 else min(read_size, tot)
    input_buffer = allocate(size, dtype=np.uint32)
    dma.recvchannel.transfer(input_buffer)
    write_reg(DMA_GRABBER,dma_garbber_en,0)
    write_reg(DMA_GRABBER,dma_garbber_size,size)
    write_reg(DMA_GRABBER,dma_garbber_en,1)
    dma.recvchannel.wait()
    data = input_buffer.copy()
    input_buffer.freebuffer()
    print(f"{size} data are read this time, and {tot-size} data remain in the FIFO.")
    return(size,data)

In [24]:
tx_data = np.arange(100, dtype=np.uint)
dma_write(tx_data)

In [ ]:
(rx_size, rx_data) = dma_read(0)
print(rx_data)